# Trabalho 1 - Coleta, Preparação e Análise de Dados

## Webscrapping em servidor local

### Objetivos da tarefa:

- Faça um crawler capaz de navegar por todas as páginas de países e baixar seus
  HTMLS.
- Faça scraping dos htmls baixados e armazene os seguintes dados dos países em
  um arquivo CSV:

      1- Nome do país (campo country)

      2- Nome da capital do país (campo capital)

      3- Nome da moeda do país (campo Currency Name)

      4- Nome do Continente (Atenção: é o nome do continente e não a sigla!)

      Salvar uma coluna extra no csv contendo um timestamp do momento no qual os dados foram obtidos.

- Faça um crawler que monitore as páginas de países e procure por atualizações.
  Caso algum registro tenha sido atualizado esse deve ser atualizado no arquivo CSV, caso
  contrário manter a versão anterior.


In [1]:
import os
import re
import csv
import requests
from datetime import datetime
from bs4 import BeautifulSoup

### Parte 1: baixando as páginas html dos países


In [4]:
# acessando a página principal
url = "http://127.0.0.1:8000/places"
requisicao = requests.get(url)

hrefs = []
if requisicao.status_code == 200:  # verificação de que a requisição foi bem sucedida
    soup = BeautifulSoup(requisicao.text, 'html.parser')
    for i in range(25): # passa por todas as páginas de index e acessa o link de cada uma
        page_url = f"{url}/default/index/{i}"
        page_requisicao = requests.get(page_url)
        page_soup = BeautifulSoup(page_requisicao.text, 'html.parser')
        # recuperando os links das páginas individuais dos países
        links = page_soup.find_all('a', href=re.compile(r"/places/default/view/.*-\d+"))
        for link in links:
            href = link.get('href')
            hrefs.append(href)
    
    # salvando os links das páginas em uma pasta, criando se já não existir
    if not os.path.exists('html_pages'):
        os.makedirs('html_pages')
    for href in hrefs:
        full_url = f"http://127.0.0.1:8000{href}"
        page_response = requests.get(full_url)
        if page_response.status_code == 200:
            file_name = href.split('/')[-1] + '.html'
            file_path = os.path.join('html_pages', file_name)
            with open(file_path, 'a', encoding='utf-8') as file:
                file.write(page_response.text)
else:
    print(f"Erro: {requisicao.status_code}")

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /places (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1085640b0>: Failed to establish a new connection: [Errno 61] Connection refused'))

### Parte 2: salvando os dados da página para um csv


In [2]:
import os
import csv
from datetime import datetime
from bs4 import BeautifulSoup

html_dir = 'html_pages'
csv_file = 'places.csv'

# mapa para converter a abreviação do continente para o nome completo
continent_dict = {
    'AF': 'Africa',
    'AN': 'Antarctica',
    'AS': 'Asia',
    'EU': 'Europe',
    'NA': 'North America',
    'OC': 'Oceania',
    'SA': 'South America'
}

# inicializando o csv
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Country', 'Capital', 'Currency Name', 'Continent', 'Timestamp'])

# itera por cada documento do diretório
for filename in os.listdir(html_dir):
    if filename.endswith('.html'):
        filepath = os.path.join(html_dir, filename)
        
        with open(filepath, 'r', encoding='utf-8') as html_file:
            soup = BeautifulSoup(html_file, 'html.parser')
            
            # procurando os dados necessários
            country = soup.find(id='places_country__row').find(class_='w2p_fw').text
            capital = soup.find(id='places_capital__row').find(class_='w2p_fw').text
            currency_name = soup.find(id='places_currency_name__row').find(class_='w2p_fw').text
            continent_abbr = soup.find(id='places_continent__row').find(class_='w2p_fw').find('a').text
            continent = continent_dict.get(continent_abbr, continent_abbr)  # Convert to full name
            
            # pegar o horário da coleta
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            
            # escrevendo num arquivo csv
            with open(csv_file, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([country, capital, currency_name, continent, timestamp])

### Parte 3: crawler que monitora a aplicação


In [3]:
with open("places.csv", "r") as arq:
    data = list(csv.reader(arq, delimiter=";"))
    comparison = False
    updated_data = [data[0]]  # Initialize with headers

    for i, line in enumerate(data[1:]):
        l = line[0].split(',')
        requisicao = requests.get(f"http://127.0.0.1:8000{hrefs[i]}")
        soup = BeautifulSoup(requisicao.content, 'html.parser')
        country = soup.find("tr", {"id": "places_country__row"}).find("td", {"class": "w2p_fw"}).text
        capital = soup.find("tr", {"id": "places_capital__row"}).find("td", {"class": "w2p_fw"}).text
        currency_name = soup.find("tr", {"id": "places_currency_name__row"}).find("td", {"class": "w2p_fw"}).text
        continent_abbr = soup.find("tr", {"id": "places_continent__row"}).find("td", {"class": "w2p_fw"}).text
        continent = continent_dict.get(continent_abbr, continent_abbr)
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        if country != l[0] or capital != l[1] or currency_name != l[2] or continent != l[3]:
            comparison = True
            updated_data.append([country, capital, currency_name, continent, timestamp])
        else:
            updated_data.append(line[0].split(','))

    if comparison:
        with open('places.csv', 'w', newline="") as new:
            writer = csv.writer(new)
            writer.writerow(['Country', 'Capital', 'Currency Name', 'Continent', 'Timestamp'])
            writer.writerows(updated_data)

NameError: name 'hrefs' is not defined